# Load points 

In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from numpy import *

In [28]:
def readstl(name): 
    STLfile = name
    f=open(STLfile,'r')

    x=[]
    y=[]
    z=[]

    for line in f:
        strarray=line.split()
        if strarray[0]=='vertex':
            x=append(x,double(strarray[1]))
            y=append(y,double(strarray[2]))
            z=append(z,double(strarray[3]))
    
    
    # calculate middle point in every triangle
    xnew = []
    ynew = []
    znew = []

    for i in range(0,len(x),3):
        xnew.append(sum(x[i:i+3])/3)
        ynew.append(sum(y[i:i+3])/3)
        znew.append(sum(z[i:i+3])/3)
    
    print('# of Sample points in %s:' %name, len(xnew))
    
    point = []
    for i in range(len(xnew)):
        point.append([xnew[i],ynew[i],znew[i]])
        
    return point

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(xnew, ynew, znew, '.')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()

# Calculate shape bins

In [33]:
import numpy as np
def shape_bins(points):
    
    print('Calculating bins')
    
    N = len(points)
    bins_all = []
    theta_Block = 12
    phi_Block = 6
    dis_Block = 5
    for point_o in points[:]:
        distances = []
        theta = []
        phi = []
        for point in points[:]:
            distance = np.sqrt((point_o[0] - point[0]) ** 2 + (point_o[1] - point[1]) ** 2 + \
                               (point_o[2] - point[2]) ** 2 )
            if distance != 0:
                distances.append(distance)
                part_distance = np.sqrt((point[0] - point_o[0]) ** 2 + (point[1] - point_o[1]) ** 2)
                
                angl = np.arccos(np.abs(point[0] - point_o[0]) / part_distance)
                
                if point[1] - point_o[1] < 0 and point[0] - point_o[0] < 0:
                    angl = angl + np.pi
                elif point[1] - point_o[1] < 0 and point[0] - point_o[0] > 0:
                    angl = np.pi * 2 - angl
                elif point[1] - point_o[1] > 0 and point[0] - point_o[0] < 0:
                    angl = angl + np.pi / 2
                theta.append(angl)  
                
                phiangl = np.arccos(part_distance / distance)
                
                if point[2] - point_o[2] < 0:
                    phiangl = -1 * angl
                
                phi.append(phiangl)  
                
        mean_dist = np.mean(distances)
        distances = distances / mean_dist

        block_lens = 0.5
        distances_log = np.log(distances / block_lens)

        for x in range(len(distances_log)):
            if distances_log[x] <= 0:
                distances_log[x] = 0
            elif distances_log[x] <= 1:
                distances_log[x] = 1
            elif distances_log[x] <= 2:
                distances_log[x] = 2
            elif distances_log[x] <= 3:
                distances_log[x] = 3
            elif distances_log[x] <= 4:
                distances_log[x] = 4
        
        for x in range(len(phi)):
            if -np.pi/2 <= phi[x] <= -np.pi/3:
                phi[x] = 0
            elif -np.pi/3 < phi[x] <= -np.pi/6:
                phi[x] = 1
            elif -np.pi/6 < phi[x] <= 0:
                phi[x] = 2
            elif 0 < phi[x] <= np.pi/6:
                phi[x] = 3
            elif np.pi/6 < phi[x] <= np.pi/3:
                phi[x] = 4
            elif np.pi/3 < phi[x] <= np.pi/2:
                phi[x] = 5
                
        for x in range(len(theta)):
            if 0 < theta[x] <= np.pi/6:
                theta[x] = 0
            elif np.pi/6 < theta[x] <= np.pi/3:
                theta[x] = 1
            elif np.pi/3 < theta[x] <= np.pi/2:
                theta[x] = 2
            elif np.pi/2 < theta[x] <= 2*np.pi/3:
                theta[x] = 3
            elif 2*np.pi/3 < theta[x] <= 5*np.pi/6:
                theta[x] = 4
            elif 5*np.pi/6 < theta[x] <= np.pi:
                theta[x] = 5
            elif np.pi < theta[x] <= 7*np.pi/6:
                theta[x] = 6
            elif 7*np.pi/6 < theta[x] <= 4*np.pi/3:
                theta[x] = 7
            elif 4*np.pi/3 < theta[x] <= 3*np.pi/2:
                theta[x] = 8
            elif 3*np.pi/2 < theta[x] <= 5*np.pi/3:
                theta[x] = 9
            elif 5*np.pi/3 < theta[x] <= 11*np.pi/6:
                theta[x] = 10
            elif 11*np.pi/6 < theta[x] <= 2*np.pi:
                theta[x] = 11

        bins = np.zeros((dis_Block, theta_Block, phi_Block))
        for x in range(len(distances_log)):
            bins[int(distances_log[x]), int(theta[x]), int(phi[x])] =  \
            bins[int(distances_log[x]), int(theta[x]), int(phi[x])] + 1
            
        bins = np.reshape(bins,[phi_Block*theta_Block*dis_Block])
        bins_all.append(bins)
    return bins_all

In [4]:
start = time.clock()

bins_all = shape_bins(point[::3])
print(bins_all[0])
print(bins_all[100])
print(bins_all[1000])

end = time.clock()
print (end-start,'seconds')

NameError: name 'point' is not defined

# calculate Cost Matrix

In [5]:
def cost_matrix(bins_A,bins_B):
    print('Calculating cost matrix')
    
    row = 0
    col = 0
    cost = np.zeros((len(bins_A), len(bins_B)))
    for bin_A in bins_A:
        col = 0
        for bin_B in bins_B:
            cost[row, col] = 0.5 * np.sum(((bin_A - bin_B) ** 2) / (bin_A + bin_B + 0.00000001))
            col = col + 1
        row = row + 1

    return cost

# Munkres to find minimum

In [16]:
import copy
from munkres import Munkres

In [49]:
def cal_munkres(cost_matrix):
    matrix = copy.deepcopy(cost_matrix)
    m = Munkres()
    indexes = m.compute(matrix)
    total = 0
    
    for row, column in indexes:
        value = cost_matrix[row][column]
        total += value

    print ('minimum cost =%d' % total)
    return total

# Main function

In [8]:
from sklearn.utils import resample

In [50]:
result = []


for i in range(1,6):

    point1 = readstl('S%d.stl'%i)
    
    for j in range(1,6):
        
        point2 = readstl('G%d.stl'%j)

        if len(point1) > len(point2):
            point2 = resample(point1, replace=True, n_samples=len(point1),random_state=123)
        else:
            point1 = resample(point2, replace=True, n_samples=len(point2),random_state=123)

        print('resample points to ',len(point1))


        point11 = point1[::300]
        point22 = point2[::300]


        bin1 = shape_bins(point11)
        bin2 = shape_bins(point22)

        cost = cost_matrix(bin1,bin2)

        total = cal_munkres(cost)
        
        result.append(total)

        print('---------------------')



# of Sample points in S1.stl: 7558
# of Sample points in G1.stl: 2934
resample points to  7558
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =288
---------------------
# of Sample points in G2.stl: 12580
resample points to  12580
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =616
---------------------
# of Sample points in G3.stl: 11040
resample points to  12580
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =634
---------------------
# of Sample points in G4.stl: 1910
resample points to  12580
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =634
---------------------
# of Sample points in G5.stl: 900
resample points to  12580
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =634
---------------------
# of Sample points in S2.stl: 14896
# of Sample points in G1.stl: 2934
resample points to  14896
Calculating bins
Calculating bins
Calculating cost matrix
minimum cost =8

In [51]:
print(result)

[288.72539485117306, 616.87476601363198, 634.55244887842014, 634.55244887842014, 634.55244887842014, 834.14382094229074, 834.14382094229074, 834.14382094229074, 834.14382094229074, 834.14382094229074, 71.499999430416665, 616.87476601363198, 634.55244887842014, 634.55244887842014, 634.55244887842014, 71.499999430416665, 616.87476601363198, 634.55244887842014, 634.55244887842014, 634.55244887842014, 71.499999430416665, 616.87476601363198, 634.55244887842014, 634.55244887842014, 634.55244887842014]
